# Ch 15: Structs and Objects

### Composite Types

We have used many of Julia’s built-in types; now we are going to define a new type. As an example, we will create a type called Point that represents a point in two-dimensional space.

There are several ways we might represent points in Julia:

* We could store the coordinates separately in two variables, $x$ and $y$.

* We could store the coordinates as elements in an array or tuple.

* We could create a new type to represent points as objects.

Creating a new type is more complicated than the other options, but it has advantages that will be apparent soon.

**A programmer-defined composite type is also called a struct.** The struct definition for a point looks like this:


In [28]:
struct Point
    x
    y
end

The header indicates that the new struct is called `Point`. The body defines the attributes or fields of the struct. The `Point` struct has two fields: $x$ and $y$.

A struct is like a factory for creating objects. To create a point, you call Point as if it were a function having as arguments the values of the fields. When Point is used as a function, it is called a **constructor**.

In [2]:
p = Point(3.0, 4.0)

Point(3.0, 4.0)

Creating a new object is called **instantiation**, and the object is an **instance** of the type.
When you print an instance, Julia tells you what type it belongs to and what the values of the atributes are.

### Structs are Immutable

You can get the values of the fields using . notation:

In [3]:
x = p.x

3.0

The expression `p.x` means, “Go to the object $p$ refers to and get the value of $x$.” In the example, we assign that value to a variable named $x$. 

Note that there is no conflict between the variable $x$ and the field $x$.

You can use dot notation as part of any expression. For example:

In [4]:
distance = sqrt(p.x^2 + p.y^2)

5.0

**Structs are, however, by default immutable,** after construction the fields can not change value:

In [6]:
p.y = 1.0

ErrorException: setfield! immutable struct of type Point cannot be changed

This may seem odd at first, but it has several advantages:

* It can be more efficient.

* It is not possible to violate the invariants provided by the type’s constructors (see Constructors).

* Code using immutable objects can be easier to reason about.

### Mutable Structs

Where required, mutable composite types can be declared with the keyword mutable struct . Here is the definition of a mutable point:

In [7]:
mutable struct MPoint
    x
    y
end

In [8]:
blank = MPoint(0.0, 0.0)
blank.x = 3.0
blank.y = 4.0

4.0

### Rectangles

Sometimes it is obvious what the fields of an object should be, but other times you have to make decisions. For example, imagine you are designing a type to represent rectangles.

There are at least two possibilities:

* You could specify one corner of the rectangle (or the center), the width, and the height.

* You could specify two opposing corners.

At this point it is hard to say whether either is better than the other, so we’ll implement the first one, just as an example.

In [9]:
"""
Represents a rectangle.
fields: width, height, corner.
"""
struct Rectangle
    width
    height
    corner
end

Rectangle

The docstring lists the fields: `width` and `height` are numbers; `corner` is a Point object that specifies the lower-left corner. 

To represent a rectangle, you have to instantiate a Rectangle object:

In [10]:
origin = MPoint(0.0, 0.0)
box = Rectangle(100.0, 200.0, origin)

Rectangle(100.0, 200.0, MPoint(0.0, 0.0))

An object that is a field of another object is **embedded**.

### Instances as Arguments

You can pass an instance as an argument in the usual way. For example:

In [29]:
function printpoint(p)
    println("($(p.x), $(p.y))")
end

printpoint (generic function with 1 method)

In [12]:
printpoint(blank)

(3.0, 4.0)


### Exercise 15-1

Write a function called `distancebetweenpoints` that takes two points as arguments and returns the distance between
them.

In [30]:
function distancebetweenpoints(p1,p2)
    d = sqrt( (p1.x-p2.x)^2 + (p1.y-p2.y)^2 )
end

distancebetweenpoints (generic function with 1 method)

In [31]:
point1 = Point(0.,0.)
point2 = Point(1.,1.)

distancebetweenpoints(point1,point2)

1.4142135623730951

**If a mutable struct object is passed to a function as an argument, the function can modify the fields of the object.** 

For example, `movepoint!` takes a mutable Point object and two numbers, $dx$ and $dy$, and adds the numbers to respectively the $x$ and the $y$ attribute of the Point:

In [13]:
function movepoint!(p, dx, dy)
    p.x += dx
    p.y += dy
    nothing
end

movepoint! (generic function with 1 method)

In [14]:
origin = MPoint(0.0, 0.0)
movepoint!(origin, 1.0, 2.0)
origin

MPoint(1.0, 2.0)

Inside the function, $p$ is an alias for origin, so when the function modifies $p$, origin changes.

Passing an immutable Point object to `movepoint!` causes an error:

In [15]:
movepoint!(p, 1.0, 2.0)

ErrorException: setfield! immutable struct of type Point cannot be changed

**You can however modify the value of a mutable attribute of an immutable object.**

For example, moverectangle! has as arguments a Rectangle object and two numbers, $dx$ and $dy$, and uses `movepoint!` to move the corner of the rectangle:

In [16]:
function moverectangle!(rect, dx, dy)
    movepoint!(rect.corner, dx, dy)
end

moverectangle! (generic function with 1 method)

Now $p$ in `movepoint!` is an alias for $rect.corner$, so when $p$ is modified, $rect.corner$ changes also:

In [32]:
box

Rectangle(100.0, 200.0, MPoint(1.0, 2.0))

In [33]:
moverectangle!(box, 1.0, 2.0)

In [34]:
box

Rectangle(100.0, 200.0, MPoint(2.0, 4.0))

**WARNING: You cannot, however, reassign a mutable attribute of an immutable object.**  Reassignment would create a new object and thus modify the immutable struct object.

In [20]:
box.corner = MPoint(1.0, 2.0)

ErrorException: setfield! immutable struct of type Rectangle cannot be changed

### Instances as Return Values

Functions can return instances. For example, `findcenter` takes a Rectangle as an argument and returns a Point that contains the coordinates of the center of the rectangle:

In [35]:
function findcenter(rect)
    Point(rect.corner.x + rect.width / 2, rect.corner.y + rect.height / 2)
end

findcenter (generic function with 1 method)

The expression `rect.corner.x` means, “Go to the object rect refers to and select the field named $corner$; then go to that object and select the field named $x$.
”
Here is an example that passes box as an argument and assigns the resulting Point to center:

In [22]:
center = findcenter(box)

Point(51.0, 102.0)

### Copying

Aliasing can make a program difficult to read because changes in one place might have unexpected effects in another place. It is hard to keep track of all the variables that might refer to a given object. Copying an object is often an alternative to aliasing. Julia provides a function called deepcopy that can duplicate any object:

In [23]:
p1 = MPoint(3.0, 4.0)

p2 = deepcopy(p1)

p1 ≡ p2, p1 == p2


(false, false)

The ≡ operator indicates that `p1` and `p2` are not the same object, which is what we expected. 

But you might have expected == to yield true because these points contain the same data. In that case, you will be disappointed to learn that for mutable objects, the default behavior of the == operator is the same as the === operator; it checks object identity, not object equivalence. That’s because for mutable composite types, Julia doesn’t know what should be considered equivalent. 

At least, not yet.

### Exercise 15-2

Create a Point instance, make a copy of it and check the equivalence and the egality of both. The result can surprise you but it explains why aliasing is a non issue for an immutable object.

In [37]:
immutable_point = Point(1,1)
deepcopy_immutable_point = deepcopy(immutable_point)

immutable_point === deepcopy_immutable_point, immutable_point == deepcopy_immutable_point


(true, true)

### Debugging

When you start working with objects, you are likely to encounter some new exceptions.

If you are not sure what type an object is, you can ask:

In [24]:
typeof(p)

Point

In [25]:
p isa Point

true

In [26]:
fieldnames(Point)

(:x, :y)

In [27]:
isdefined(p, :x)

true

The first argument can be any object; the second argument is a symbol, : followed by the name of the field.